<a href="https://colab.research.google.com/github/serenach33/aiffel_ex/blob/main/E1_RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration 1: Rock-Paper-Scissors Image Classification

---

#### Model
* Keras Sequential API

#### Datasets
* Created my own datasets from [Teachable Machine](https://teachablemachine.withgoogle.com/)
* Gathered more datasets from the Aiffel mates

#### Model evaluation
* Loss and Accuracy
* Confusion Matrix
* Classification report

#### Goals

1. Get higher accuracy than the first result which was 0.37, hopefully above 0.80
2. Import more datasets
3. Use `sklearn.model_selection.train_test_split`
4. Plot Confusion Matrix and Classification report
5. Validate my hypothesis: If test datasets are highly qualified than the train sets, then the accuracy will increase.

---






## Importing Dependencies

laying the foundation to build my model on

In [3]:
import os
import glob
import random
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
